訓練模型

In [1]:
#%%
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import load_model

import joblib
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import os

#載入訓練資料
device = 'L1'
SourceData = pd.read_csv(f"..//Data//MergedSorted//{device}_Merged_Sorted.csv")
SourceData.dropna(inplace=True)
SourceData.reset_index(inplace=True)

In [7]:
input_features_model_1 = to_predict_features_model_1 = input_features_model_2 = [
    'Avg_Temperature(°C)',
    'Avg_Humidity(%)',
    'Avg_Sunlight(Lux)',

    'Min_Temperature(°C)',
    'Min_Humidity(%)',
    'Min_Sunlight(Lux)',
    'Min_Power(mW)',
    
    'Max_Temperature(°C)',
    'Max_Humidity(%)',
    'Max_Sunlight(Lux)',
    'Max_Power(mW)'
]


to_predict_features_model_2 = [
    'Avg_Power(mW)',
    ]

In [ ]:
def create_dataset(data, LookBackNum):
    X = []
    y = []

    #設定每i-12筆資料(X_train)就對應到第i筆資料(y_train)
    for i in range(LookBackNum,len(data)):
        X.append(data[i-LookBackNum:i, :])
        y.append(data[i, :])

    return np.array(X), np.array(y)
  
#設定LSTM往前看的筆數和預測筆數
n_timesteps = LookBackNum = 6 #LSTM往前看的筆數


#迴歸分析 選擇要留下來的資料欄位
#(發電量)
Regression_X_train = SourceData[to_predict_features_model_2].values
Regression_y_train = SourceData[['Avg_Power(mW)']].values


#LSTM 選擇要留下來的資料欄位
AllOutPut = SourceData[to_predict_features_model_1].values
n_features = len(to_predict_features_model_1)


#正規化
LSTM_StandardModel = StandardScaler().fit(AllOutPut)
AllOutPut_Scaled = LSTM_StandardModel.transform(AllOutPut)
X_train, y_train = create_dataset(AllOutPut_Scaled, LookBackNum=LookBackNum)


# Reshaping
#(samples 是訓練樣本數量,timesteps 是每個樣本的時間步長,features 是每個時間步的特徵數量)
X_train = np.reshape(X_train,(X_train.shape[0], n_timesteps, n_features))

In [20]:
#%%
#============================建置&訓練「LSTM模型」============================
#建置LSTM模型

regressor = Sequential ()

regressor.add(LSTM(
    units = 200, 
    return_sequences = True, 
    input_shape = (n_timesteps, n_features),
    activation='relu'
    ))

regressor.add(Dropout(0.1))

regressor.add(LSTM(
    units = 64, 
    activation='relu',
    return_sequences = False, 
    ))



# output layer
regressor.add(Dense(units = n_features))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 6, 200)            169600    
                                                                 
 dropout (Dropout)           (None, 6, 200)            0         
                                                                 
 lstm_19 (LSTM)              (None, 64)                67840     
                                                                 
 dense_6 (Dense)             (None, 11)                715       
                                                                 
Total params: 238,155
Trainable params: 238,155
Non-trainable params: 0
_________________________________________________________________


In [21]:
#開始訓練
regressor.fit(X_train, y_train, epochs = 100, batch_size = 128)

Epoch 1/100
74/74 [==============================] - 3s 24ms/step - loss: 0.2847
Epoch 2/100
74/74 [==============================] - 2s 22ms/step - loss: 0.1058
Epoch 3/100
74/74 [==============================] - 2s 22ms/step - loss: 0.0913
Epoch 4/100
74/74 [==============================] - 2s 23ms/step - loss: 0.0852
Epoch 5/100
74/74 [==============================] - 2s 23ms/step - loss: 0.0818
Epoch 6/100
74/74 [==============================] - 2s 23ms/step - loss: 0.0801
Epoch 7/100
74/74 [==============================] - 2s 23ms/step - loss: 0.0802
Epoch 8/100
74/74 [==============================] - 2s 24ms/step - loss: 0.0777
Epoch 9/100
74/74 [==============================] - 2s 24ms/step - loss: 0.0767
Epoch 10/100
74/74 [==============================] - 2s 24ms/step - loss: 0.0761
Epoch 11/100
74/74 [==============================] - 2s 23ms/step - loss: 0.0770
Epoch 12/100
74/74 [==============================] - 2s 22ms/step - loss: 0.0760
Epoch 13/100
74/74 [=====

In [22]:
#保存模型
regressor.save(f'..//Model//WheatherLSTM_{device}.h5')
print('Model Saved')

Model Saved


In [48]:
#載入模型
regressor = load_model(f'..//Model//WheatherLSTM_{device}.h5')
print('Model Loaded Successfully')

Model Loaded Successfully


In [71]:
y_pred = regressor.predict(X_train)

295/295 [==============================] - 1s 4ms/step


In [49]:
def SeqNumber2Datetime(seq_number: int) -> datetime:
    seq_number = str(seq_number)
    datetime_str = seq_number[:12]
    return datetime.strptime(datetime_str, "%Y%m%d%H%M") 

def Datetime2SeqNumber(dt: datetime, device_id) -> int:
    base_seq_number = dt.strftime("%Y%m%d%H%M")
    seq_number_with_device = f"{base_seq_number}{device_id:02}"
    return seq_number_with_device

In [50]:
SourceData = pd.read_csv(f"..//Data//MergedSorted//{device}_Merged_Sorted.csv")
SourceData

,DateTime,SeqNumber,Device_ID,Year,Month,Day,Hour,Minute,Avg_WindSpeed(m/s),Avg_Pressure(hpa),...,Std_Min_Temperature(°C),Std_Min_Humidity(%),Std_Min_Sunlight(Lux),Std_Min_Power(mW),Std_Max_WindSpeed(m/s),Std_Max_Pressure(hpa),Std_Max_Temperature(°C),Std_Max_Humidity(%),Std_Max_Sunlight(Lux),Std_Max_Power(mW)
0,2024-01-01 06:30:00,20240101063001,1.0,2024.0,1.0,1.0,6.0,30.0,0.0,1016.54,...,-1.277347,0.546684,-0.844341,-0.526245,-0.499413,0.022279,-1.308157,0.498119,-0.895384,-0.632017
1,2024-01-01 06:40:00,20240101064001,1.0,2024.0,1.0,1.0,6.0,40.0,0.0,1016.59,...,-1.287759,0.546684,-0.841386,-0.526245,-0.499413,0.022629,-1.308157,0.498119,-0.890886,-0.631981
2,2024-01-01 06:50:00,20240101065001,1.0,2024.0,1.0,1.0,6.0,50.0,0.0,1016.66,...,-1.277347,0.554346,-0.835507,-0.526170,-0.499413,0.022920,-1.308157,0.505943,-0.885882,-0.631909
3,2024-01-01 07:00:00,20240101070001,1.0,2024.0,1.0,1.0,7.0,0.0,0.0,1016.73,...,-1.277347,0.554346,-0.829109,-0.526071,-0.499413,0.023270,-1.308157,0.502031,-0.881711,-0.631855
4,2024-01-01 07:10:00,20240101071001,1.0,2024.0,1.0,1.0,7.0,10.0,0.0,1016.77,...,-1.277347,0.539022,-0.823068,-0.525997,-0.499413,0.023794,-1.298099,0.509854,-0.874230,-0.631693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23439,2024-11-30 16:20:00,20241130162001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23440,2024-11-30 16:30:00,20241130163001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23441,2024-11-30 16:40:00,20241130164001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23442,2024-11-30 16:50:00,20241130165001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
target_positions = []
for timestamp, row in SourceData.iterrows():
    if row.isna().any():
        target_position = SourceData.index.get_loc(timestamp)
        if target_position >= LookBackNum :
            target_positions.append(target_position)

In [52]:
len(target_positions)

14003

In [ ]:
PredictedData = SourceData.copy()
TestData = pd.read_csv('..//Data/TestData//upload(no answer).csv')
TestData = TestData[TestData['序號'] % 10 == 1]
to_predict_sequmber = TestData['序號'].to_list()
indices = PredictedData[PredictedData['SeqNumber'].isin(to_predict_sequmber)][to_predict_features_model_1].index.to_list()

index = 365
X = PredictedData.loc[index-LookBackNum:index-1][to_predict_features_model_1]
X = LSTM_StandardModel.transform(X)
X

c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[-0.95169219,  0.46603063,  0.37116394, -1.06465711,  0.20368217,
         0.05936453, -0.1002847 , -0.82458499,  0.62734486,  0.388398  ,
         0.44874661],
       [-0.67394109, -0.13733424,  0.46080787, -0.74321933, -0.24606517,
        -0.17142648, -0.35586912, -0.63996929,  0.00302258,  0.48479795,
         0.53597618],
       [-0.67789484, -0.24906847, -0.01155299, -0.71308453, -0.24225375,
        -0.17030725, -0.34564479, -0.64968591, -0.25291078,  0.64372317,
         0.82976813],
       [-0.70853642, -0.21285117,  0.90852794, -0.77335412, -0.34897346,
        -0.01417665, -0.25852401, -0.5816696 , -0.14433299,  0.8667303 ,
         1.23902378],
       [-0.43375063, -0.51992767,  0.94448844, -0.48205113, -0.60814989,
        -0.20247871, -0.39455512, -0.38733728, -0.46618858,  0.96998149,
         1.2493763 ],
       [-0.31909181, -0.69099664,  1.30278764, -0.31128731, -0.70724676,
         0.94703699,  0.80062124, -0.30960436, -0.71436638,  1.11568757,
         1.420

In [ ]:
for index in indices:
    X = PredictedData.loc[index-LookBackNum : index-1][to_predict_features_model_1]
    X = LSTM_StandardModel.transform(X)
    X = np.reshape(X,(1, n_timesteps, n_features))
    pred = regressor.predict(X)
    pred = pred[0]
    PredictedData.loc[index, to_predict_features_model_1] = pred

1/1 [==============================] - 0s 28ms/step
Avg_Temperature(°C)     -1.5173
Avg_Humidity(%)        0.951715
Avg_Sunlight(Lux)     -0.815515
Min_Temperature(°C)    -1.58397
Min_Humidity(%)        1.120342
Min_Sunlight(Lux)     -0.892934
Min_Power(mW)         -0.672599
Max_Temperature(°C)   -1.471924
Max_Humidity(%)        0.876587
Max_Sunlight(Lux)      -0.66958
Max_Power(mW)         -0.853543
Name: 365, dtype: object
1/1 [==============================] - 0s 22ms/step
Avg_Temperature(°C)   -2.699006
Avg_Humidity(%)       -1.374691
Avg_Sunlight(Lux)     -0.579726
Min_Temperature(°C)   -2.977031
Min_Humidity(%)        -1.91857
Min_Sunlight(Lux)     -0.440156
Min_Power(mW)         -0.198879
Max_Temperature(°C)   -2.227317
Max_Humidity(%)       -1.130022
Max_Sunlight(Lux)      0.165189
Max_Power(mW)          1.207128
Name: 366, dtype: object
1/1 [==============================] - 0s 28ms/step


c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Avg_Temperature(°C)    -3.28029
Avg_Humidity(%)       -1.127719
Avg_Sunlight(Lux)     -1.707311
Min_Temperature(°C)   -3.562776
Min_Humidity(%)       -2.439385
Min_Sunlight(Lux)     -1.640243
Min_Power(mW)         -0.356777
Max_Temperature(°C)   -3.030564
Max_Humidity(%)       -1.149386
Max_Sunlight(Lux)     -1.489658
Max_Power(mW)         -0.336334
Name: 367, dtype: object
1/1 [==============================] - 0s 17ms/step
Avg_Temperature(°C)   -1.802872
Avg_Humidity(%)       -1.660439
Avg_Sunlight(Lux)     -0.497896
Min_Temperature(°C)   -1.440544
Min_Humidity(%)       -1.541087
Min_Sunlight(Lux)      0.009398
Min_Power(mW)         -0.257575
Max_Temperature(°C)   -1.177308
Max_Humidity(%)       -1.238982
Max_Sunlight(Lux)     -0.311287
Max_Power(mW)          0.360223
Name: 368, dtype: object
1/1 [==============================] - 0s 29ms/step
Avg_Temperature(°C)   -2.648281
Avg_Humidity(%)       -1.666198
Avg_Sunlight(Lux)     -1.441931
Min_Temperature(°C)   -2.942439
Min_Humidity(%

c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 37ms/step
Avg_Temperature(°C)   -3.247985
Avg_Humidity(%)       -2.198967
Avg_Sunlight(Lux)     -1.686184
Min_Temperature(°C)   -3.534915
Min_Humidity(%)       -2.520443
Min_Sunlight(Lux)     -1.274806
Min_Power(mW)          0.051104
Max_Temperature(°C)   -3.789938
Max_Humidity(%)       -1.905755
Max_Sunlight(Lux)     -1.323073
Max_Power(mW)         -0.487501
Name: 372, dtype: object
1/1 [==============================] - 0s 11ms/step
Avg_Temperature(°C)   -3.303368
Avg_Humidity(%)       -2.280503
Avg_Sunlight(Lux)     -1.817029
Min_Temperature(°C)   -3.562451
Min_Humidity(%)       -2.508287
Min_Sunlight(Lux)     -1.433934
Min_Power(mW)         -0.022213
Max_Temperature(°C)   -3.867783
Max_Humidity(%)       -1.977399
Max_Sunlight(Lux)     -1.471227
Max_Power(mW)         -0.587426
Name: 373, dtype: object
1/1 [==============================] - 0s 33ms/step
Avg_Temperature(°C)   -3.282999
Avg_Humidity(%)       -2.239527
Avg_Sunlight(Lux)      -1.

...

Avg_Temperature(°C)   -3.363956
Avg_Humidity(%)       -2.254933
Avg_Sunlight(Lux)     -1.769506
Min_Temperature(°C)   -3.642089
Min_Humidity(%)       -2.533277
Min_Sunlight(Lux)     -1.362844
Min_Power(mW)          0.023191
Max_Temperature(°C)   -3.923576
Max_Humidity(%)       -1.932725
Max_Sunlight(Lux)     -1.391797
Max_Power(mW)         -0.521185
Name: 6197, dtype: object
1/1 [==============================] - 0s 17ms/step
Avg_Temperature(°C)   -3.363961
Avg_Humidity(%)       -2.254922
Avg_Sunlight(Lux)     -1.769488
Min_Temperature(°C)   -3.642099
Min_Humidity(%)        -2.53328
Min_Sunlight(Lux)     -1.362822
Min_Power(mW)          0.023207
Max_Temperature(°C)   -3.923579
Max_Humidity(%)       -1.932712
Max_Sunlight(Lux)     -1.391774
Max_Power(mW)         -0.521167
Name: 6198, dtype: object
1/1 [==============================] - 0s 21ms/step
Avg_Temperature(°C)   -3.363979
Avg_Humidity(%)       -2.254926
Avg_Sunlight(Lux)     -1.769505
Min_Temperature(°C)   -3.642111
Min_Humidity

c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 23ms/step
Avg_Temperature(°C)   -3.363987
Avg_Humidity(%)       -2.254922
Avg_Sunlight(Lux)     -1.769502
Min_Temperature(°C)   -3.642119
Min_Humidity(%)       -2.533262
Min_Sunlight(Lux)     -1.362846
Min_Power(mW)          0.023202
Max_Temperature(°C)    -3.92361
Max_Humidity(%)        -1.93271
Max_Sunlight(Lux)     -1.391788
Max_Power(mW)         -0.521178
Name: 6202, dtype: object
1/1 [==============================] - 0s 16ms/step
Avg_Temperature(°C)   -3.363989
Avg_Humidity(%)       -2.254922
Avg_Sunlight(Lux)     -1.769504
Min_Temperature(°C)   -3.642121
Min_Humidity(%)       -2.533261
Min_Sunlight(Lux)     -1.362849
Min_Power(mW)          0.023202
Max_Temperature(°C)   -3.923612
Max_Humidity(%)        -1.93271
Max_Sunlight(Lux)      -1.39179
Max_Power(mW)          -0.52118
Name: 6203, dtype: object
1/1 [==============================] - 0s 22ms/step
Avg_Temperature(°C)   -3.363989
Avg_Humidity(%)       -2.254922
Avg_Sunlight(Lux)     -1

c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\envs\py_scikit\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 21ms/step
Avg_Temperature(°C)    -3.36399
Avg_Humidity(%)       -2.254922
Avg_Sunlight(Lux)     -1.769504
Min_Temperature(°C)   -3.642121
Min_Humidity(%)        -2.53326
Min_Sunlight(Lux)     -1.362849
Min_Power(mW)          0.023202
Max_Temperature(°C)   -3.923614
Max_Humidity(%)       -1.932709
Max_Sunlight(Lux)     -1.391789
Max_Power(mW)          -0.52118
Name: 6206, dtype: object
